<a target="_blank" rel="noopener noreferrer" href="https://colab.research.google.com/github/ccaudek/ds4psy_2023/blob/main/16a_xarray.ipynb">![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)</a>

(xarray-notebook)=
# Un introduzione a xarray

L'obiettivo di questo capitolo è analizzare in dettaglio il flusso di lavoro all'interno del contesto dell'inferenza bayesiana, una volta che sia stata ottenuta la stima della distribuzione a posteriori mediante l'utilizzo della tecnica MCMC.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import pymc as pm
import arviz as az
import scipy.stats as stats
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
%config InlineBackend.figure_format = 'retina'

RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

pd.set_option('display.precision', 2)

from aquarel import load_theme
theme = load_theme("minimal_light")
theme.apply()
sns.set_palette("colorblind")

In [3]:
ary = np.random.default_rng().normal(size=(4, 100, 7, 24))
da_dims = xr.DataArray(ary, dims=["chain", "draw", "person", "time"])
da_dims

<xarray.DataArray (chain: 4, draw: 100, person: 7, time: 24)>
array([[[[-1.47891573e+00,  1.21109758e-01, -1.14157509e+00, ...,
           3.47654767e-01, -5.45718066e-01, -9.79851696e-01],
         [-2.11433557e-01, -1.29281966e+00, -2.85995678e-01, ...,
           7.45351644e-03, -1.18488523e+00, -2.12893472e+00],
         [-1.65029420e+00,  1.35008575e+00,  3.16381844e-01, ...,
          -1.13237439e+00,  1.30558433e-01,  1.29039583e-01],
         ...,
         [ 2.31833796e-01,  1.55504672e+00,  9.02618769e-01, ...,
          -5.38545585e-01,  6.42493711e-02,  2.37161544e-01],
         [-1.30706994e+00,  4.55658575e-01, -2.69988527e-01, ...,
           9.57792626e-01, -1.48238219e+00,  5.84721362e-01],
         [-2.06086511e+00, -6.57118467e-01,  5.88047899e-01, ...,
          -3.21381911e-01,  1.19309984e-01,  5.14850524e-01]],

        [[-1.02859907e-01, -1.71570634e-01, -3.66291085e-01, ...,
           4.74316334e-01, -5.25724700e-01,  4.18236019e-01],
         [ 6.10415464e-01, -4.96408237e-01, -4.06634317e-01, ...,
           4.66217486e-01,  5.85626821e-01, -6.26418881e-01],
         [-5.79588288e-01, -8.96706403e-01,  4.26089570e-01, ...,
          -2.24734486e-01,  1.15290213e+00,  1.69460652e-02],
...
         [ 6.17744981e-01, -1.02651638e+00, -7.38539184e-01, ...,
          -6.13117231e-01, -1.35805262e+00,  1.47436100e+00],
         [ 4.08629379e-02, -3.78587846e-01, -2.25758031e+00, ...,
          -1.04642673e+00,  4.69435677e-01, -1.39936099e-01],
         [-1.38061506e-01,  1.36101124e-01,  1.15824885e+00, ...,
          -8.56736289e-01, -3.13924406e-01,  1.43346566e+00]],

        [[ 7.09578631e-01,  9.16172604e-01, -1.23036530e+00, ...,
           1.20046101e+00,  1.12937134e+00, -1.11322152e+00],
         [-4.45773961e-01, -1.11938682e+00,  2.57073914e+00, ...,
          -3.51391729e-01, -6.75926100e-01,  1.56173380e-03],
         [ 5.37465140e-01,  5.20119786e-01,  1.56311061e+00, ...,
          -3.99669453e-01,  5.74158242e-03,  1.36748169e+00],
         ...,
         [ 5.76601892e-01,  1.88998142e+00,  7.91679719e-01, ...,
           3.68894599e-01, -1.83259394e+00,  2.60562167e-01],
         [ 6.23005664e-01,  9.03786858e-02, -6.15552153e-02, ...,
          -2.93252603e-01,  6.03005958e-01,  4.78919807e-01],
         [ 6.38658103e-01, -2.06778747e+00, -2.68240236e-01, ...,
           1.26201427e-01, -2.48617020e-01, -8.18384186e-01]]]])
Dimensions without coordinates: chain, draw, person, time

In [4]:
da_dims.mean(dim=("chain", "draw"))

<xarray.DataArray (person: 7, time: 24)>
array([[ 1.65799705e-01, -4.76480916e-02,  5.75848182e-02,
         1.04520089e-02,  2.95892244e-02,  5.53754850e-02,
         7.88559614e-02, -3.85891277e-02,  2.43171951e-03,
         5.23244087e-02, -3.05784179e-02,  1.27009833e-02,
         7.64846875e-02,  3.79033605e-02, -7.35756663e-02,
         5.76628265e-02, -6.07544126e-02, -1.80181091e-03,
         2.08555317e-02, -3.63270406e-03,  2.95753977e-02,
        -2.64410124e-02, -8.15561099e-02, -1.20823477e-01],
       [ 2.64571064e-05, -7.92193612e-02,  6.03965936e-02,
        -2.09464670e-02, -9.22923733e-03,  1.13734939e-02,
        -6.71339059e-02,  5.09801272e-04, -4.42418599e-02,
        -5.21187117e-02,  5.99645545e-03, -4.98931424e-02,
         8.19980500e-03,  2.82223250e-02, -5.10946410e-02,
        -6.66331787e-02, -2.64095617e-02,  5.05821291e-02,
         3.79645264e-02,  5.51847980e-02,  2.37666922e-02,
         3.34841975e-02,  7.04783004e-03, -4.89046023e-02],
       [-3.78031744e-02, -2.32379989e-02,  6.30201898e-02,
        -2.84607708e-02,  1.53490794e-02,  8.68297285e-04,
         4.98655191e-02, -1.73051058e-02, -2.37609002e-02,
         9.52617138e-02,  1.97983199e-02, -6.46214423e-03,
...
        -1.90093521e-02,  1.40601276e-02,  8.78302196e-02,
         2.85708529e-02, -6.95894177e-03, -8.15380347e-03,
        -7.44183508e-02,  1.14960801e-02,  2.04024223e-02,
        -2.97425169e-02, -2.24564277e-02,  8.42546285e-03],
       [ 5.01700889e-02, -1.82369328e-02, -4.57324916e-02,
        -2.82970762e-02, -1.07773812e-01,  3.68137249e-02,
         6.04207927e-03,  1.84820456e-02,  2.04009758e-02,
         3.46169217e-02, -3.30426476e-02, -6.42441337e-02,
        -1.37224615e-01,  3.98079154e-02,  1.68560474e-03,
         7.34831672e-02,  3.66618360e-02, -3.63851961e-02,
         1.59457963e-01,  3.37901248e-02,  5.72847232e-02,
        -2.33369712e-02, -5.13798813e-02, -2.40065751e-02],
       [-4.79126968e-02,  2.70637896e-03, -6.77943058e-02,
         5.35399054e-02, -4.19956609e-02, -6.93449242e-02,
        -6.09320434e-02, -3.37226424e-02, -4.97377293e-02,
         2.17021051e-02, -5.46928814e-03, -7.28772700e-02,
         6.14328224e-02,  7.22710116e-02,  7.39203265e-02,
        -2.99595110e-02, -7.99422594e-03,  6.61022032e-04,
        -5.20233750e-02,  5.10323524e-02,  4.59897196e-02,
         8.70280694e-02,  3.13077838e-02, -3.52362250e-02]])
Dimensions without coordinates: person, time

## Watermark

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Sat Jul 15 2023

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.12.0

seaborn   : 0.12.2
pymc      : 5.6.1
pandas    : 1.5.3
scipy     : 1.10.1
arviz     : 0.16.0
matplotlib: 3.7.1
numpy     : 1.25.0

Watermark: 2.4.3

